## For Creating Results from Newly Collected and Labeled Data

In [2]:
import pandas as pd
df_new=pd.read_csv('./labeled_banners.csv')

In [3]:
## Manual labeling must be added to the labeled_banners.csv for this task. This is done in provided dataset, but would need to be done for additional collected data. 
##These denote if the button is hiddden in the text or formatted as a link (a=accept,m=manage,r=reject=h=hidden in text, l=link)

from collections import Counter
df_new['Note']=df_new['Note'].astype(str)
Counter(df_new.Note)

Counter({'nan': 1127,
         'ml': 334,
         'ml, mh': 61,
         'rl': 55,
         'ml, rl': 15,
         'rl, ml': 9,
         'rl, ml, mh': 7,
         'rl, rh': 6,
         'ml, al': 2,
         'ml, rl, al': 2,
         'rl, al': 2,
         'ml, mh, rl': 1,
         'ml ,mh': 1,
         'ml ,rl': 1})

In [4]:
##choices with no accept all or reject all button, only granular choices
## For the provided dataset, you must remove this one URL for explaining salience analyses as it provides only a single button/choice menu
## With only one button, no comparison can be made under website-level fixed effects.
df_new=df_new[df_new.shortened_url!='ed.ac.uk']
print(len(df_new))

1621


In [5]:
##min max scaling of all buttons - NOTE that we discount the "Zero" buttons
print(df_new[df_new.accept_salience_max!=0]['accept_salience_avg'].min(),df_new.accept_salience_avg.max())
print(df_new[df_new.manage_salience_max!=0]['manage_salience_avg'].min(),df_new.manage_salience_avg.max())
print(df_new[df_new.reject_salience_max!=0]['reject_salience_avg'].min(),df_new.reject_salience_avg.max())
print(df_new[df_new.x_salience_max!=0]['x_salience_avg'].min(),df_new.x_salience_avg.max())

print(df_new[df_new.accept_salience_max!=0]['accept_salience_max'].min(),df_new.accept_salience_max.max())
print(df_new[df_new.manage_salience_max!=0]['manage_salience_max'].min(),df_new.manage_salience_max.max())
print(df_new[df_new.reject_salience_max!=0]['reject_salience_max'].min(),df_new.reject_salience_max.max())
print(df_new[df_new.x_salience_max!=0]['x_salience_max'].min(),df_new.x_salience_max.max())



df_new['accept_salience']=((df_new['accept_salience_avg']-6.516304347826087)/(223.32955936352508-6.516304347826087))+((df_new['accept_salience_max']-15)/(255-15))
df_new['reject_salience']=((df_new['reject_salience_avg']-6.516304347826087)/(223.32955936352508-6.516304347826087))+((df_new['reject_salience_max']-15)/(255-15))
df_new['manage_salience']=((df_new['manage_salience_avg']-6.516304347826087)/(223.32955936352508-6.516304347826087))+((df_new['manage_salience_max']-15)/(255-15))
df_new['x_salience']=((df_new['x_salience_avg']-6.516304347826087)/(223.32955936352508-6.516304347826087))+((df_new['x_salience_max']-15)/(255-15))

17.455460244143847 223.32955936352508
9.611863313990971 212.03860414394765
7.088961038961039 219.75686813186812
6.516304347826087 210.20974235104669
66 255
35 255
24 255
15 255


In [6]:
##Based on manual labels (link, hidden text, reject button in corner, etc.) and bounding boxes, you can then collect button characteristics

import pandas as pd
import json
from PIL import Image
import numpy as np
new_rows = []
for index, row in df_new.iterrows():
    if index % 50==0:
        print(index)
    domain=row['shortened_url']
    if row.EUIP==True:
        image = Image.open(f"./CookieBanners_ICWSM_EU/{domain}.png")
        ineu='true'
    if row.EUIP==False:
        image = Image.open(f"./CookieBanners_ICWSM_NY/{domain}.png")
        ineu='false'
    gray_image = image.convert('L')
    image_array = np.array(gray_image)
    gray_image.close()
    image.close()
    data = row['label']
    dict_values = json.loads(data)
    for i in dict_values:
        #extract banner chars - brightness
        if i['rectanglelabels'][0]=='banner':
            banner_x=i['x']
            banner_y=i['y']
            banner_width=i['width']
            banner_height=i['height']
            banner_brightness=(image_array[int(banner_y/100*i['original_height']):int((banner_y+banner_height)/100*i['original_height']),int(banner_x/100*i['original_width']):int((banner_x+banner_width)/100*i['original_width'])].mean())
        else:
            continue
    for i in dict_values:
        #extract button chars
        if i['rectanglelabels'][0]!='banner':
            buttontype=i['rectanglelabels'][0]
            note=row['Note']
            hidden=False
            link=False
            #link or hidden binary indicators
            if buttontype=='accept':
                if 'al' in note:
                    link=True
                if 'ah' in note:
                    hidden=True
            elif buttontype=='manage':
                if 'ml' in note:
                    link=True
                if 'mh' in note:
                    hidden=True
            elif buttontype=='reject':
                if 'rl' in note:
                    link=True
                if 'rh' in note:
                    hidden=True
            ##get brightness of button
            gscale_brightness=(image_array[int(i['y']/100*i['original_height']):int((i['y']+i['height'])/100*i['original_height']),int(i['x']/100*i['original_width']):int((i['x']+i['width'])/100*i['original_width'])].mean())
            new_row = {
                ## diff URL for NY/EU visits
                'url': row['shortened_url']+'_'+ineu, 
                ## banner label the button is from
                'label':row['Final_label'],
                'salience':row[f'{buttontype}_salience'],
                'button': buttontype,            
                'banner_size': banner_width*banner_height,              
                'button_size': i['width']*i['height'] ,
               'button_size_p':(i['width']*i['height'])/(banner_width*banner_height),
                ## is button in corner (either reject or "X")
                'corner':(i['rectanglelabels'][0]=='x')|((i['rectanglelabels'][0]=='reject')&(row['Final_label']=='x')),
                ## choice menu indicator
                'choices':((i['rectanglelabels'][0]=='manage')&(row['Final_label'] in ['choices','choices xa','full choices xa','full choices'])),
                'brightness':gscale_brightness,
                'banner_brightness':banner_brightness,
                ##  diff between background and button brightness
                'contrast':abs(gscale_brightness-banner_brightness),
                ## distance from center of page
                'banner_distance':np.sqrt(((banner_x+banner_width/2-50)**2)+((banner_y+banner_height/2-50)**2)),
                ## distance from center of page
                'button_distance':np.sqrt(((i['x']+i['width']/2-50)**2)+((i['y']+i['height']/2-50)**2)),
                ## distance from center of banner
                'button_banner_distance':np.sqrt(((i['x']+i['width']/2-banner_x-banner_width/2)**2)+((i['y']+i['height']/2-banner_y-banner_height/2)**2)),
                'link':link,
                'hidden':hidden
                
           
            }

            new_rows.append(new_row)
        else:
            continue

            

df = pd.DataFrame(new_rows)
df


0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600


,url,label,salience,button,banner_size,button_size,button_size_p,corner,choices,brightness,banner_brightness,contrast,banner_distance,button_distance,button_banner_distance,link,hidden
0,7sun.eu_false,full,1.271183,reject,2519.924531,107.068752,0.042489,False,False,226.541451,233.040706,6.499255,13.431818,13.956601,16.001882,False,False
1,7sun.eu_false,full,1.166932,manage,2519.924531,91.599631,0.036350,False,False,228.977040,233.040706,4.063666,13.431818,6.345063,11.382436,False,False
2,7sun.eu_false,full,1.455062,accept,2519.924531,73.817151,0.029293,False,False,82.192783,233.040706,150.847923,13.431818,30.858032,31.466551,False,False
3,20minutes.fr_false,full,1.561759,reject,5408.817981,52.996929,0.009798,False,False,157.700338,238.712259,81.011921,0.102177,14.530205,14.531409,True,True
4,20minutes.fr_false,full,1.928757,accept,5408.817981,143.007586,0.026440,False,False,116.142909,238.712259,122.569350,0.102177,35.490390,35.402115,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4605,zwoenitz.de_true,full,1.413480,reject,3576.384668,110.835142,0.030991,False,False,146.484532,209.706838,63.222306,10.281334,14.868663,25.148367,False,False
4606,zwoenitz.de_true,full,1.161385,manage,3576.384668,56.198100,0.015714,False,False,241.143168,209.706838,31.436330,10.281334,21.726274,26.553932,False,False
4607,marriott.com_true,full,1.681858,accept,3810.497239,130.361168,0.034211,False,False,89.373601,226.419492,137.045891,9.687201,36.894695,30.891168,False,False
4608,marriott.com_true,full,1.772631,reject,3810.497239,127.891714,0.033563,False,False,84.866042,226.419492,141.553449,9.687201,28.398136,20.089604,False,False


In [7]:
Counter(df.groupby('url')[['label']].first()['label'])

Counter({'full': 755,
         'manage': 346,
         'full xa': 142,
         'full-m': 97,
         'full choices': 70,
         'x': 69,
         'manage xa': 59,
         'choices': 41,
         'full-m xa': 24,
         'full choices xa': 12,
         'choices xa': 6})

In [8]:
df.to_csv('./button_salience_characteristics.csv')

## Analysis and Replicating Paper Results

In [9]:
import pandas as pd
## Load the button salience characteristics file to replicate results
##Or use your created one from previous section if using new data
df=pd.read_csv('./button_salience_characteristics.csv')

from sklearn.preprocessing import MinMaxScaler

# Normalize 'salience' to be between 0 and 1 for interpretability
scaler = MinMaxScaler()
df['salience_normalized'] = scaler.fit_transform(df[['salience']])

In [10]:
df

,url,label,salience,button,banner_size,button_size,button_size_p,corner,choices,brightness,banner_brightness,contrast,banner_distance,button_distance,button_banner_distance,link,hidden,salience_normalized
0,7sun.eu_false,full,1.271183,reject,2519.924531,107.068752,0.042489,False,False,226.541451,233.040706,6.499255,13.431818,13.956601,16.001882,False,False,0.634463
1,7sun.eu_false,full,1.166932,manage,2519.924531,91.599631,0.036350,False,False,228.977040,233.040706,4.063666,13.431818,6.345063,11.382436,False,False,0.582176
2,7sun.eu_false,full,1.455062,accept,2519.924531,73.817151,0.029293,False,False,82.192783,233.040706,150.847923,13.431818,30.858032,31.466551,False,False,0.726687
3,20minutes.fr_false,full,1.561759,reject,5408.817981,52.996929,0.009798,False,False,157.700338,238.712259,81.011921,0.102177,14.530205,14.531409,True,True,0.780201
4,20minutes.fr_false,full,1.928757,accept,5408.817981,143.007586,0.026440,False,False,116.142909,238.712259,122.569350,0.102177,35.490390,35.402115,False,False,0.964268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4605,zwoenitz.de_true,full,1.413480,reject,3576.384668,110.835142,0.030991,False,False,146.484532,209.706838,63.222306,10.281334,14.868663,25.148367,False,False,0.705832
4606,zwoenitz.de_true,full,1.161385,manage,3576.384668,56.198100,0.015714,False,False,241.143168,209.706838,31.436330,10.281334,21.726274,26.553932,False,False,0.579394
4607,marriott.com_true,full,1.681858,accept,3810.497239,130.361168,0.034211,False,False,89.373601,226.419492,137.045891,9.687201,36.894695,30.891168,False,False,0.840437
4608,marriott.com_true,full,1.772631,reject,3810.497239,127.891714,0.033563,False,False,84.866042,226.419492,141.553449,9.687201,28.398136,20.089604,False,False,0.885963


In [11]:
from sklearn.preprocessing import StandardScaler

categorical_vars = ['corner', 'choices', 'link', 'hidden','label','url','button']
continuous_vars = [col for col in df.columns if col not in categorical_vars + ['salience','salience_normalized']]

# Standardize only continuous (non-binary) variables
scaler = StandardScaler()
df[continuous_vars] = scaler.fit_transform(df[continuous_vars])


In [16]:
## If you want to include banner vars (there are webpage (i.e., banner-level) fixed effects included)

import statsmodels.formula.api as smf


df['corner'] = df['corner'].astype(int)
df['choices'] = df['choices'].astype(int)
df['link'] = df['link'].astype(int)
df['hidden'] = df['hidden'].astype(int)


formula = "salience ~ banner_size + button_size + button_size_p + brightness + contrast + " \
          "banner_distance + button_distance + button_banner_distance + corner + choices + link+ hidden + C(url)"


model = smf.ols(formula, data=df).fit()


print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               salience   R-squared:                       0.836
Model:                            OLS   Adj. R-squared:                  0.746
Method:                 Least Squares   F-statistic:                     9.319
Date:                Wed, 23 Jul 2025   Prob (F-statistic):               0.00
Time:                        17:51:30   Log-Likelihood:                 1609.1
No. Observations:                4610   AIC:                             43.71
Df Residuals:                    2979   BIC:                         1.054e+04
Df Model:                        1630                                         
Covariance Type:            nonrobust                                         
                                                       coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------

In [44]:
## including non-linear vars

import statsmodels.formula.api as smf
 

df['corner'] = df['corner'].astype(int)
df['choices'] = df['choices'].astype(int)
df['link'] = df['link'].astype(int)
df['hidden'] = df['hidden'].astype(int)


formula = "salience_normalized ~ banner_size + button_size + brightness + contrast + " \
          "banner_distance + button_distance + button_banner_distance + corner + choices + link+ hidden + C(url)+I(button_size**2)"


model = smf.ols(formula, data=df).fit()


print(model.summary())

                             OLS Regression Results                            
Dep. Variable:     salience_normalized   R-squared:                       0.835
Model:                             OLS   Adj. R-squared:                  0.745
Method:                  Least Squares   F-statistic:                     9.262
Date:                 Mon, 21 Apr 2025   Prob (F-statistic):               0.00
Time:                         15:42:01   Log-Likelihood:                 4778.4
No. Observations:                 4610   AIC:                            -6295.
Df Residuals:                     2979   BIC:                             4202.
Df Model:                         1630                                         
Covariance Type:             nonrobust                                         
                                                       coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------

In [12]:
## INCLUDED IN PAPER
import statsmodels.formula.api as smf


df['corner'] = df['corner'].astype(int)
df['choices'] = df['choices'].astype(int)
df['link'] = df['link'].astype(int)
df['hidden'] = df['hidden'].astype(int)

formula = "salience_normalized ~ button_size  + brightness + contrast + " \
          "  button_distance + button_banner_distance + corner + link+ choices+ hidden + C(url)"


model = smf.ols(formula, data=df).fit()

print(model.summary())

                             OLS Regression Results                            
Dep. Variable:     salience_normalized   R-squared:                       0.835
Model:                             OLS   Adj. R-squared:                  0.745
Method:                  Least Squares   F-statistic:                     9.266
Date:                 Thu, 21 Aug 2025   Prob (F-statistic):               0.00
Time:                         11:24:23   Log-Likelihood:                 4777.5
No. Observations:                 4610   AIC:                            -6295.
Df Residuals:                     2980   BIC:                             4196.
Df Model:                         1629                                         
Covariance Type:             nonrobust                                         
                                                       coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------

In [11]:
## Only using brightness/contrast

import statsmodels.formula.api as smf

# Ensure 'corner' and 'choices' are integers
df['corner'] = df['corner'].astype(int)
df['choices'] = df['choices'].astype(int)
df['link'] = df['link'].astype(int)
df['hidden'] = df['hidden'].astype(int)

# Define the formula for the regression model
formula = "salience_normalized ~ contrast + brightness+ C(url)"

# Fit the model using OLS
model = smf.ols(formula, data=df).fit()

# Print the summary (fixed effects won't be listed explicitly)
print(model.summary())

                             OLS Regression Results                            
Dep. Variable:     salience_normalized   R-squared:                       0.770
Model:                             OLS   Adj. R-squared:                  0.646
Method:                  Least Squares   F-statistic:                     6.175
Date:                 Wed, 14 May 2025   Prob (F-statistic):               0.00
Time:                         12:05:39   Log-Likelihood:                 4013.1
No. Observations:                 4610   AIC:                            -4780.
Df Residuals:                     2987   BIC:                             5665.
Df Model:                         1622                                         
Covariance Type:             nonrobust                                         
                                                       coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------

In [12]:
##only fixed effects
formula = "salience_normalized ~ C(url)"

model = smf.ols(formula, data=df).fit()

print(model.summary())

                             OLS Regression Results                            
Dep. Variable:     salience_normalized   R-squared:                       0.518
Model:                             OLS   Adj. R-squared:                  0.256
Method:                  Least Squares   F-statistic:                     1.980
Date:                 Wed, 14 May 2025   Prob (F-statistic):           2.88e-58
Time:                         12:06:13   Log-Likelihood:                 2302.9
No. Observations:                 4610   AIC:                            -1364.
Df Residuals:                     2989   BIC:                             9069.
Df Model:                         1620                                         
Covariance Type:             nonrobust                                         
                                                       coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------